<a href="https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/distilbert_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
!pip install transformers

     |████████████████████████████████| 317kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 47.4MB/s 
     |████████████████████████████████| 860kB 48.8MB/s 
     |████████████████████████████████| 645kB 43.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=60135dbf1248bf3a94375ddbeec791a41ffa74f0346a7f96d2c25d41f0608ce1
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
import transformers as ppb

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [5]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [6]:
df[:5][0].values

array(['a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films',
       'apparently reassembled from the cutting room floor of any given daytime soap',
       "they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science fiction elements of bug eyed monsters and futuristic women in skimpy clothes",
       'this is a visually stunning rumination on love , memory , history and the war between art and commerce',
       "jonathan parker 's bartleby should have been the be all end all of the modern office anomie films"],
      dtype=object)

In [0]:
batch_1 = df[:2000]

In [8]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [9]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

100%|██████████| 267967963/267967963 [00:05<00:00, 51387886.41B/s]


In [10]:
df[0][:5]

0    a stirring , funny and finally transporting re...
1    apparently reassembled from the cutting room f...
2    they presume their audience wo n't sit still f...
3    this is a visually stunning rumination on love...
4    jonathan parker 's bartleby should have been t...
Name: 0, dtype: object

In [11]:
tokenizer.encode("a visually stunning rumination on love", add_special_tokens=True)

[101, 1037, 17453, 14726, 19379, 12758, 2006, 2293, 102]

In [12]:
tokenizer.decode(102)

'[ S E P ]'

In [22]:
tokenized_5[0], tokenized_5[1], tokenized_5[1999]

KeyError: ignored

In [0]:
sentence = tokenizer.encode("a visually stunning rumination on love", add_special_tokens=True)

In [0]:
tokenized_5 = df[0][:5].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [0]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [21]:
tokenized[1999]

[101,
 1996,
 3185,
 2003,
 25757,
 2011,
 1037,
 24466,
 16134,
 2008,
 1005,
 1055,
 2074,
 6388,
 2438,
 2000,
 7344,
 3686,
 1996,
 7731,
 4378,
 2096,
 13060,
 18856,
 17322,
 2094,
 2000,
 15015,
 10271,
 4641,
 102]

In [14]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

59

In [0]:
padded = [i + [0]*(max_len-len(i)) for i in tokenized.values]

In [16]:
np.array(padded).shape

(2000, 59)

In [0]:
input_ids = torch.tensor(np.array(padded))  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
# input_ids = torch.tensor([sentence])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.

with torch.no_grad():
    last_hidden_states = model(input_ids)

In [18]:
last_hidden_states[0].shape

torch.Size([2000, 59, 768])

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
labels = batch_1[1]

In [0]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [22]:
from sklearn.model_selection import GridSearchCV
parameters = {'C': np.linspace(0.001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Sp

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.00...2105e+01, 1.57903158e+01,
       2.10534211e+01, 2.63165263e+01, 3.15796316e+01, 3.68427368e+01,
       4.21058421e+01, 4.73689474e+01, 5.26320526e+01, 5.78951579e+01,
       6.31582632e+01, 6.84213684e+01, 7.36844737e+01, 7.89475789e+01,
       8.42106842e+01, 8.94737895e+01, 9.47368947e+01, 

In [24]:
grid_search.best_params_

{'C': 5.264105263157894}

In [25]:
grid_search.best_score_

0.7986666666666666

In [0]:
lr_clf = LogisticRegression(C= 78.94757894736841)

In [26]:

lr_clf = LogisticRegression(C= 5.264)
# lr_clf.fit(train_features, train_labels)

scores = cross_val_score(lr_clf, train_features, train_labels)
print("Logistic Regression classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression classifier score: 0.799 (+/- 0.02)


In [27]:
lr_clf = LogisticRegression(C=5.264)
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.81

In [18]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, train_features, train_labels)
print("Random forest classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random forest classifier score: 0.683 (+/- 0.04)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
from sklearn.svm import SVC
svc_regressor = SVC()

scores = cross_val_score(svc_regressor, train_features, train_labels)
print("SVM classifier average score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in v

SVM classifier average score: 0.525 (+/- 0.00)


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
# est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')
gbc = GradientBoostingClassifier()

scores = cross_val_score(gbc, train_features, train_labels, cv=5)
print("Gradient Boosting classifier accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



Gradient Boosting classifier accuracy: 0.765 (+/- 0.03)


In [26]:
from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(1000, 300), learning_rate='adaptive', verbose=False)
neural_network.fit(train_features, train_labels)
neural_network.score(test_features, test_labels)
# scores = cross_val_score(neural_network, train_features, train_labels)
# print("Neural network regressor score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


0.792

In [27]:
from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(2000), learning_rate='adaptive', verbose=False)
neural_network.fit(train_features, train_labels)
neural_network.score(test_features, test_labels)


0.784

In [22]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.497 (+/- 0.02)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [10]:
from google.colab import files
uploaded = files.upload()

Saving 500_with_vectors.csv to 500_with_vectors.csv


In [0]:
df = pd.read_csv('500_with_vectors.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [14]:
df

,sentence,label,split,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,new,1,train,0.233748,-0.222612,0.270256,-0.100287,-0.300720,-0.325703,0.098270,0.133757,0.152318,-0.164728,0.059485,0.067821,0.154655,0.054548,0.070726,0.094673,0.025708,0.182993,-0.061887,-0.094307,0.200075,0.183657,0.371726,0.157151,-0.303662,-0.008491,0.088826,0.353364,0.249312,0.121814,-0.034527,0.101452,-0.247590,-0.023922,0.117177,0.410878,0.230009,...,0.071863,-0.152217,0.314776,0.201787,-0.221472,-0.323454,0.174078,0.200405,-0.265705,-0.298790,0.070027,0.126764,-0.550447,-0.102870,-0.299570,-0.029691,0.404164,-0.019329,0.571036,0.190634,0.486656,0.551100,-0.386007,0.553028,-1.911065,-0.058988,0.323301,-0.237405,-0.305501,0.051247,0.153581,-0.213971,0.118741,0.043862,0.268415,-0.302601,0.125339,-0.080951,0.739329,-0.114249
1,splash,1,train,0.228766,-0.213051,0.269835,-0.105115,-0.287857,-0.327904,0.094252,0.142503,0.132024,-0.151163,0.041531,0.040875,0.138779,0.078235,0.106438,0.086404,0.027244,0.184633,-0.068678,-0.089802,0.220715,0.194699,0.361071,0.150054,-0.308772,-0.010095,0.081059,0.379280,0.270694,0.124302,-0.009915,0.101524,-0.274123,-0.001704,0.120562,0.417391,0.221819,...,0.083690,-0.127704,0.329485,0.195718,-0.229896,-0.313575,0.182254,0.225073,-0.247905,-0.318933,0.059433,0.151987,-0.526197,-0.111069,-0.309602,-0.072512,0.383994,-0.030114,0.589379,0.152922,0.489214,0.548229,-0.347108,0.546480,-1.840378,-0.056731,0.301595,-0.239712,-0.268412,0.031907,0.147502,-0.197948,0.107488,0.052756,0.274319,-0.344441,0.114706,-0.092937,0.752795,-0.112522
2,a splash,1,train,0.137208,-0.313407,0.226146,-0.145098,-0.333741,-0.297151,0.106797,0.168328,0.101784,-0.184324,0.039666,0.053048,0.102834,0.053712,0.111207,0.077887,0.088367,0.120244,-0.025574,-0.164704,0.202982,0.149291,0.322254,0.122163,-0.323041,-0.001579,0.060177,0.414324,0.282133,0.062029,-0.011203,0.122134,-0.283472,-0.024014,0.099437,0.403131,0.209398,...,0.038631,-0.081034,0.296371,0.220180,-0.179793,-0.227891,0.124991,0.270777,-0.279706,-0.341659,0.121395,0.146233,-0.450034,-0.127239,-0.290212,-0.045135,0.410151,-0.031487,0.610341,0.229661,0.499684,0.526126,-0.319516,0.539215,-2.143089,-0.110817,0.372068,-0.199167,-0.221969,0.033992,0.137732,-0.277083,0.125730,0.019359,0.279625,-0.349366,0.141246,-0.061664,0.745740,-0.145962
3,greater,1,train,0.244372,-0.217179,0.269788,-0.094656,-0.301732,-0.294661,0.091070,0.110606,0.174087,-0.179891,0.052742,0.062356,0.154614,0.066787,0.078596,0.127853,0.024617,0.193381,-0.070514,-0.097330,0.234804,0.186447,0.371675,0.142241,-0.281845,0.007323,0.096840,0.356626,0.263522,0.109556,-0.027977,0.111564,-0.236160,-0.023637,0.137945,0.407442,0.244270,...,0.084874,-0.162486,0.300559,0.175735,-0.212014,-0.340711,0.197919,0.228845,-0.266319,-0.290397,0.042575,0.110049,-0.557211,-0.127587,-0.319923,-0.082219,0.399482,-0.043312,0.566519,0.173379,0.497346,0.537554,-0.367088,0.539756,-1.802676,-0.053023,0.306297,-0.248355,-0.305621,0.067563,0.146111,-0.167810,0.103711,0.085224,0.280750,-0.313768,0.111990,-0.080343,0.747839,-0.102310
4,a splash even greater,1,train,0.116057,-0.320084,0.254907,-0.138818,-0.379596,-0.254312,0.114805,0.132261,0.120034,-0.178364,0.047495,0.056912,0.107906,0.044411,0.092156,0.090518,0.086257,0.132753,-0.039860,-0.192983,0.247434,0.179486,0.325996,0.141971,-0.282370,0.046241,0.103692,0.376202,0.291703,0.058385,-0.024277,0.120918,-0.214091,-0.055446,0.115729,0.371713,0.215142,...,0.006576,-0.110749,0.246992,0.248963,-0.131180,-0.226813,0.143871,0.223728,-0.269268,-0.295310,0.118405,0.100665,-0.425310,-0.131608,-0.270887,-0.027740,0.406834,-0.056763,0.584286,0.262561,0.473144,0.512692,-0.325140,0.511781,-2.274318,-0.107610,0.371740,-0.189074,-0.268028,0.013178,0.124603,-0.250946,0.128571,0.057087,0.300056,-0.298960,0.138990,-0.0